In [1]:
import os
import json
import csv
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df_food = pd.read_csv('./data/레시피_기본정보.csv', error_bad_lines=False, encoding="cp949")
df_material = pd.read_csv('./data/레시피_재료정보.csv', error_bad_lines=False, encoding="cp949")
df_process = pd.read_csv('./data/레시피_과정정보.csv', error_bad_lines=False, encoding="cp949")

b'Skipping line 67: expected 5 fields, saw 7\nSkipping line 99: expected 5 fields, saw 8\nSkipping line 110: expected 5 fields, saw 6\nSkipping line 152: expected 5 fields, saw 6\nSkipping line 159: expected 5 fields, saw 6\nSkipping line 166: expected 5 fields, saw 6\nSkipping line 392: expected 5 fields, saw 6\nSkipping line 396: expected 5 fields, saw 6\nSkipping line 510: expected 5 fields, saw 7\nSkipping line 516: expected 5 fields, saw 10\nSkipping line 522: expected 5 fields, saw 15\nSkipping line 575: expected 5 fields, saw 6\nSkipping line 608: expected 5 fields, saw 12\nSkipping line 674: expected 5 fields, saw 9\nSkipping line 675: expected 5 fields, saw 6\nSkipping line 706: expected 5 fields, saw 10\nSkipping line 728: expected 5 fields, saw 15\nSkipping line 738: expected 5 fields, saw 15\nSkipping line 739: expected 5 fields, saw 18\nSkipping line 794: expected 5 fields, saw 7\nSkipping line 805: expected 5 fields, saw 13\nSkipping line 867: expected 5 fields, saw 17\nS

In [4]:
dict_food = {} #전체 재료에 대한 정보

dict_main = {}  #주재료
dict_spyce = {} #부재료
dict_side = {}  #양념


for index, row in df_material.iterrows() :
    #-----------------------------------------------------
    #각 Dictionary 공간에 해당되는 재료 삽입
    if row['레시피 코드'] in dict_food:
        dict_food[row['레시피 코드']] += row['재료명'] + ','
    else :
        dict_food[row['레시피 코드']] =  row['재료명'] + ','
        #빈 Dicotionary 공간 생성
        dict_main[row['레시피 코드']] =  ''
        dict_side[row['레시피 코드']] =  ''
        dict_spyce[row['레시피 코드']] = ''
        
    if row['재료타입명'] == '주재료' :
        dict_main[row['레시피 코드']] += row['재료명'] + ','
        
    elif row['재료타입명'] == '부재료' :
        dict_side[row['레시피 코드']] += row['재료명'] + ','
        
    elif row['재료타입명'] == '양념' :
        dict_spyce[row['레시피 코드']] +=  row['재료명'] + ','
    #------------------------------------------------------

    
#-----------------------------------------------------------
#pandas 기본정보에 정보를 입력하기 위해 각 공간 할당
df_food['재료'] = ''
df_food['주재료'] = ''
df_food['부재료'] = ''
df_food['양념'] = ''

for key, value in dict_food.items():
    food_value = value.split(',')
    food_value = food_value[:-1] #마지막 열 제거 ','
    dict_food[key] = food_value
    
    #주재료-----------------------------------------
    try :
        main_value = dict_main[key].split(',')
    except :
        main_value = ['']
    #부재료-----------------------------------------
    try :
        side_value = dict_side[key].split(',')
    except :
        side_value = ['']
    #양념-------------------------------------------
    try :
        spyce_value = dict_spyce[key].split(',')
    except :
        spyce_value = ['']
    #--------------------------------------------- 
    
    #마지막 리스트에 존재하는 ''요소 삭제
    dict_main[key] =  main_value[:-1]
    dict_side[key] =  side_value[:-1]
    dict_spyce[key] = spyce_value[:-1]

food_list = []
main_list = []
side_list = []
spyce_list = []
#----------------------------------------------------
#신규 COLUMN에 포함되는 값들을 LIST에 저장
#----------------------------------------------------
for index, row in df_food.iterrows() :
    food_list.append(dict_food[row['레시피 코드']] )
    main_list.append(dict_main[row['레시피 코드']] )
    side_list.append(dict_side[row['레시피 코드']] )
    spyce_list.append(dict_spyce[row['레시피 코드']] )
    
#신규 COLUMN 작성
df_food['재료'] = food_list
df_food['주재료'] = main_list
df_food['부재료'] = side_list
df_food['양념'] = spyce_list


In [5]:
all_material_list = df_material['재료명'].unique() #모든 요리들의 재료 리스트
vector_index_list = []  #요리가 가지고 있는 재료들을 비교한 리스트 값
vector_set_list = []    #각 백터들의 설정값

for index ,row in df_food.iterrows():
    #해당 요리에 포함되는 모든 재료
    food_material = row['재료']
    food_index = []
    
    #주재료, 부재료, 양념에 대한 각 가중치 Value
    each_material = [row['주재료'],row['부재료'],row['양념']]
    each_weight_list = [6,2,2]#가중치 임의값
    
    #벡터 공간
    vector = [0]* len(all_material_list) 
    
    #요리 재료 순회, 각 재료의 재료 타입에 따른 가중치 입력
    for data in food_material : 
        data = data.strip()
        try :
            index = np.where(all_material_list == data)[0][0]
            food_index.append(index)
            for i in range(3) :
                if data in each_material[i] :
                    vector[index] = each_weight_list[i]
        except :
            print('잘못된 재료 입력입니다 : {}'.format(data))
    vector_index_list.append(food_index)
    vector_set_list.append(vector)
    
    
df_food['재료 인덱스'] = vector_index_list
df_food['VECTOR'] = vector_set_list


잘못된 재료 입력입니다 : 청피망 각각
잘못된 재료 입력입니다 : 래디쉬


In [6]:
#메뉴와 메뉴에 대한 비교후 점수 리턴
def compare_menu(menu1_vector, menu2_vector) :
    score = 0
    for index in range(len(menu1_vector)) :
        if menu1_vector[index] != 0 and menu2_vector[index] != 0 :
            score += menu1_vector[index]
        elif menu1_vector[index] == 0 and menu2_vector[index] != 0 :
            score -= menu2_vector[index] // 2 #부족한 재료에 대해 -처리(임의 상수값)
    return score

#레시피 메뉴 코드에 대해 메뉴 추천
def process_menu_code(df, code, menu_cnt) :
    cur_menu_vector = df[df['레시피 코드'] == code]['VECTOR'].item()
    comp_dict = {}
    for index, row in df.iterrows():
        if row['레시피 코드'] == code :
            continue
        comp_menu_vector = row['VECTOR']
        score = compare_menu(cur_menu_vector, comp_menu_vector)
        
        #점수 리스트 
        comp_dict[row['레시피 코드']] = score
    
    cur_menu_name = df[df['레시피 코드']== code]['레시피 이름'].item()
    print('현재 메뉴 : {}'.format(cur_menu_name))
    
    #Dictionary의 큰값순서대로 정렬된 리스트를 만들어, Menu cnt만큼 추천 요리 표시
    comp_list = sorted(comp_dict.items(), key=lambda x: x[1], reverse= True)
    for index in range(menu_cnt):
        #comp_list[index][0] = Key, comp_list[index][1] = score
        recommand_menu = df[df['레시피 코드']== comp_list[index][0]]['레시피 이름'].item()
        print('추천 메뉴 {} :'.format(index+1) ,recommand_menu)    


#유저 입력에 대해 메뉴 추천
def process_user_input(df,user_list,menu_cnt) :
    #유저가 입력한 재료 리스트 표시
    print('유저 입력:', user_list)
    
    #유저 입력에 대한 백터 공간 설정
    user_vector = [0]* len(all_material_list)
    for data in user_list :
        try :
            index = np.where(all_material_list == data)[0][0]
            user_vector[index] = 1
        except :
            print('잘못된 재료 입력입니다 : {}'.format(data))
            
    #유저가 가지고 있는 재료에 대해 각 요리들을 비교
    comp_dict = {}
    for index, row in df.iterrows():
        comp_food = row['VECTOR']
        
        #유저의 입력에는 주재료, 부재료에 대한 가중치가 없어서, 반대로 비교를 수행한다.
        score = compare_menu(comp_food, user_vector) 
        comp_dict[row['레시피 코드']] = score
    
    #Dictionary의 큰값순서대로 정렬된 리스트를 만들어, Menu cnt만큼 추천 요리 표시
    comp_list = sorted(comp_dict.items(), key=lambda x: x[1], reverse= True)
    for index in range(menu_cnt):
        #comp_list[index][0] = Key, comp_list[index][1] = score
        recommand_menu = df[df['레시피 코드']== comp_list[index][0]]['레시피 이름'].item()
        print('추천 메뉴 {} :'.format(index+1) ,recommand_menu)    
    return 


In [8]:
temp_input = ['쇠고기', '간장', '양파', '대파','돼지고기']

process_user_input(df_food,temp_input,5)
process_menu_code(df_food, 41, 5)

유저 입력: ['쇠고기', '간장', '양파', '대파', '돼지고기']
추천 메뉴 1 : 롤케비지(양배추말이)
추천 메뉴 2 : 매운가지볶음
추천 메뉴 3 : 쌀국수
추천 메뉴 4 : 한국식타코샐러드
추천 메뉴 5 : 제육배추찜
현재 메뉴 : 두부다시마말이
추천 메뉴 1 : 다시마볶음
추천 메뉴 2 : 두부탕
추천 메뉴 3 : 미소된장국
추천 메뉴 4 : 다시마냉국
추천 메뉴 5 : 참치죽
